In [ ]:
# default_exp models.live_data

In [ ]:
import statsapi as mlb
import datetime as dt
from pydantic import (
    #BaseModel,
    root_validator,
    validator,
    conint,
    conlist
)
from MLB_DataDevTools.models.base_models import *
from enum import Enum
from typing import Any,List,Optional,Union
from typing_extensions import Literal
from pydantic import BaseModel as PydanticBaseModel

class BaseModel(PydanticBaseModel):
    class Config:
        extra='forbid'

In [ ]:
import pandas as pd
from MLB_DataDevTools.database import create_mlb_engine
engine = create_mlb_engine()

In [ ]:
games = pd.read_sql('game',engine.connect())

pk = games['game_pk'].sample().iloc[0]

game = mlb.get('game',{'gamePk':pk})

liveData = game['liveData']

liveData.keys()

plays = liveData['plays']['allPlays']

play = plays[20]


## Plays

In [ ]:
play.keys()

In [ ]:
play['result']

In [ ]:
event_types = mlb.get('meta',{'type':'eventTypes'})
event_types[0]

In [ ]:
import warnings 

class EventType(BaseModel):
    code: str
    plateAppearance: bool
    hit: bool
    baseRunningEvent: bool
    description: str
    
    @root_validator(pre=True)
    def get_event_type(cls,values):
        code = values['code']
        values.update(cls.Config.event_types.get(code,{}))
        return values

    class Config:
        event_types = {
            pe.pop('code'):pe for pe in 
            mlb.get('meta',{'type':'eventTypes'})
        }
    def __init__(self,code:str,**kwargs):
        
        super().__init__(code=code)
        for k,v in kwargs.items():
            if kwargs[k]!=self.dict()[k]:
                warnings.warn(f"""
                    according to the values stored in the the `EventType`
                    model config, 
                    the value for {k} should be {self.dict()[k]},
                    but you passed the value: {v}.
                """)

In [ ]:
EventType(code='double',plateAppearance=True)

In [ ]:
EventType('double')

In [ ]:
EventType(code='double',plateAppearance=False)

In [ ]:
class ModelWithEventType(BaseModel):
    """
    Some responses from the API only have a string to represent the `eventType`. 
    This Model takes those strings and turns them into the `EventType` Model
    """
    def __init__(self,eventType:str = None,**kwargs):
        super().__init__(
            eventType={'code':eventType},
            **kwargs
        )

## Play Result

In [ ]:
#exporti 

class PlayResult(ModelWithEventType):
    type: str
    event: str
    eventType: EventType
    description: str
    rbi: conint(ge=0)
    awayScore: conint(ge=0)
    homeScore: conint(ge=0)
    

In [ ]:
play['result']

In [ ]:
PlayResult(**play['result']).dict()

## Play About

In [ ]:
#exporti

class HalfInning(str,Enum):
    top='top'
    bottom='bottom'

class PlayAbout(BaseModel):
    atBatIndex: conint(ge=0)
    halfInning: HalfInning
    isTopInning:bool
    inning: conint(ge=0)
    startTime: dt.datetime
    endTime: dt.datetime
    isComplete: bool
    isScoringPlay: bool
    hasReview: bool
    hasOut: bool
    captivatingIndex: int

In [ ]:
play['about']

In [ ]:
PlayAbout(**play['about']).dict()

## Count

In [ ]:
play['count']

In [ ]:
#exporti 

class Count(BaseModel):
    balls: conint(ge=0,le=4)
    strikes: conint(ge=0,le=3)
    outs: conint(ge=0,le=3)

In [ ]:
Count(**play['count'])

## Matchup

In [ ]:
play['matchup']

In [ ]:
{p['matchup']['splits']['menOnBase'] for p in plays}

In [ ]:
#exporti

class MenOnBase(str,Enum):
    Empty='Empty'
    Men_On='Men_On'
    RISP='RISP'

class BatterSplit(str,Enum):
    vs_RHP='vs_RHP'
    vs_LHP='vs_LHP'
class PitcherSplit(str,Enum):
    vs_RHB='vs_RHB'
    vs_LHB='vs_LHB'
class Splits(BaseModel):
    batter: BatterSplit
    pitcher: PitcherSplit
    menOnBase: MenOnBase

In [ ]:
#exporti

class Matchup(BaseModel):
    batter: PersonBase
    batSide: PlayerHandedness
    pitcher: PersonBase
    pitchHand: PlayerHandedness
    postOnFirst: Optional[PersonBase] = None
    postOnSecond: Optional[PersonBase] = None
    postOnThird: Optional[PersonBase] = None
    batterHotColdZones: conlist(Any,max_items=0) # I want to see if this ever comes back with items
    pitcherHotColdZones: conlist(Any,max_items=0)
    splits: Splits

In [ ]:
Matchup(**play['matchup']).dict()

## Runners

In [ ]:
class Base(str,Enum):
    first_base='1B'
    second_base='2B'
    third_base='3B'
    score='score'

In [ ]:
[p['runners'][0]['movement'] for p in plays if p['about']['isScoringPlay']][5:10]

In [ ]:
movement_reasons = set()
n=0
while n < 10:
    pk = games['game_pk'].sample().iloc[0]

    game = mlb.get('game',{'gamePk':pk})

    liveData = game['liveData']

    plays = liveData['plays']['allPlays']


    {movement_reasons.add(runners['details']['movementReason'])
     for play in plays for runners in play['runners']}
    n+=1

In [ ]:
movement_reasons

> note: how can I make a custom field that is an Enum with descriptions? 

In [ ]:
class MovementReason(str,Enum):
    advanced_on_force='r_adv_force'
    advanced_on_play='r_adv_play'
    advanced_on_throw='r_adv_throw'
    thrown_out='r_thrown_out'
    force_out='r_force_out'
    defensive_indifference='r_defensive_indiff'
    pickoff_error_1b='r_pickoff_error_1b'
    pickoff_error_2b='r_pickoff_error_2b'
    pickoff_error_3b='r_pickoff_error_3b'
    stolen_base_2b='r_stolen_base_2b'

In [ ]:
class Movement(BaseModel):
    originBase: Optional[Base] = None
    outBase: Optional[Base] = None
    start: Optional[Base] = None
    end: Optional[Base] = None
    isOut: bool
    outNumber: Optional[conint(ge=0,le=3)] = None

In [ ]:
play['runners'][0]['details']

In [ ]:
class RunnerDetails(ModelWithEventType):
    event: str
    eventType: EventType
    movementReason: Optional[str]
    runner: PersonBase
    responsiblePitcher: Optional[PersonBase] = None
    isScoringEvent: bool
    rbi: bool
    earned: bool
    teamUnearned: bool
    playIndex: int

In [ ]:
play['runners'][0]['credits']

In [ ]:
from typing_extensions import TypedDict

In [ ]:

class Credit(BaseModel):
    player: TypedDict(
        'player',
        id=int,
        link=str
    )
    position: PositionBase
    credit: str

In [ ]:
play['runners'][0]

In [ ]:
class Runner(BaseModel):
    movement: Movement
    details: RunnerDetails
    credits: List[Credit]

In [ ]:
play['runners'][0]

In [ ]:
Runner(**play['runners'][0])

## Play Event

In [ ]:
play=plays[0]
play_events = play['playEvents']

pitches = [pitch for play in plays for pitch in play['playEvents'] if pitch['isPitch']]
not_pitches = [pitch for play in plays for pitch in play['playEvents'] if not pitch['isPitch']]

In [ ]:
from pydantic.color import Color

In [ ]:
pitches[0]['details']

In [ ]:
not_pitches[0]['details']

In [ ]:
pitches[5].keys()

### Pitch Details

In [ ]:
class PitchCall(BaseModel):
    code:str
    description:str

class PitchType(BaseModel):
    code:str
    description:str
    
    #maybe add some descriptions here, too? like mentioned before in the runners section?

In [ ]:
import pydantic
pydantic.version.VERSION

In [ ]:

class PitchDetails(PydanticBaseModel):
    
    call: PitchCall
    description:str
    code: str
    ballColor: Color
    trailColor: Color
    isInPlay: bool
    isStrike: bool
    hasReview: bool

In [ ]:
pitches[0].keys()

In [ ]:
pitches[0]['count']

In [ ]:
play['count']

In [ ]:
pitches[0]['pitchData']

In [ ]:
class PitchCoordinatesBase(BaseModel):
    x: float
    y: float

class PitchCoordinates(PitchCoordinatesBase):
    aY: float
    aZ: float
    pfxX: float
    pfxZ: float
    pX: float
    pZ: float
    vX0: float
    vY0: float
    vZ0: float
    x0: float
    y0: float
    z0: float
    aX: float


In [ ]:
class PitchBreaks(BaseModel):
    breakAngle: float
    breakLength: float
    breakY: float
    spinRate: float
    spinDirection: float

In [ ]:
class PitchData(BaseModel):
    
    startSpeed: Optional[float]=None
    endSpeed: Optional[float]=None
    strikeZoneTop: float
    strikeZoneBottom: float
    coordinates: Union[PitchCoordinates,PitchCoordinatesBase]
    breaks: Union[PitchBreaks,dict]
    zone: Optional[int]
    typeConfidence: Optional[float]=None
    plateTime: Optional[float]=None
    extension: Optional[float]=None

In [ ]:
pitches[0]['pitchData']

In [ ]:
pitch_data = PitchData(**pitches[0]['pitchData'])
pitch_data

In [ ]:
pitches[0].keys()

In [ ]:
pitches[0]['type']

In [ ]:
{x['type'] for x in pitches}

### Play Event // Pitch

In [ ]:
#mlb.get('meta',{'type':'eventTypes'})

Reduce duplication between Action, pickoff, pitch details

In [ ]:
class PlayEventBase(BaseModel):
    count: Count
    index: int
    startTime: dt.datetime
    endTime: Optional[dt.datetime]=None
    isPitch:bool
    isSubstitution:bool=False
    isBaseRunningPlay:bool=False
    type: Literal['pitch','action','pickoff']

class ActionDetails(ModelWithEventType):
    description: str
    event: str
    eventType: EventType
    awayScore:int
    homeScore:int
    isScoringPlay:bool
    hasReview:bool     
    
class Action(PlayEventBase):
    details:ActionDetails
    player:Optional[PersonBase]=None
    position:Optional[PositionBase]=None
    umpire:Optional[PersonBase]=None
    battingOrder:Optional[str]=None
    replacedPlayer:Optional[PersonBase]=None
    base:Optional[int]=None

class PickoffDetails(BaseModel):
    description:str
    code:int
    hasReview:bool
    fromCatcher:bool
        
class Pickoff(PlayEventBase):
    details:PickoffDetails
    playId:Optional[str]=None
    actionPlayId:Optional[str]=None

class Pitch(PlayEventBase):
    details:PitchDetails
    pitchData:PitchData
    playId:str
    pitchNumber: int
    type:Literal['pitch']='pitch'

In [ ]:
Pitch(**pitches[0])

In [ ]:
Pickoff(**[x for x in not_pitches if x['type']=='pickoff'][0])

In [ ]:
Action(**[x for x in not_pitches if x['type']=='action'][-3])

In [ ]:
[x for x in not_pitches if x['type']=='action'][-2]

## Play Model

In [ ]:
play.keys()

In [ ]:
play['atBatIndex']

In [ ]:
class Play(BaseModel):
    result:PlayResult
    about:PlayAbout
    count:Count
    matchup:Matchup
    pitchIndex:List[int]
    actionIndex:List[int]
    runnerIndex:List[int]
    runners:List[Runner]
    playEvents:List[Union[Pitch,Pickoff,Action]]
    atBatIndex:int
    playEndTime:dt.datetime

In [ ]:
[Play(**p) for p in plays]